# readme

> I may have mentioned before, somewhere on ltu, the incident about
> three decades ago when someone told me there was no point in designing
> programming languages because in a few years all computers would be
> self-programming, and I told them if computers wrote programs they'd
> want a really good programming language.

- [John Shutt (2018-11-30)](http://lambda-the-ultimate.org/node/5559)
- [The Kernel Programming Language](https://web.cs.wpi.edu/~jshutt/kernel.html)

i want to use a vau calculus lisp for generative model stuff: as an
action space for agents, as a prompt language for image models etc.

i think the lisp should be pure except for a couple of effects:

1. `define` to write code without using something like one giant `let`
2. `assert` for latent type information.
3. `shift`/`reset` for external control by an effect handler

for other effects, the interpreter should be used like a generator and
produce a stream of data representing actions. a handler should
collect the actions, execute them, and give the result to a
continuation.

# version 1

- ✅ fexprs
- ✅ continuation-passing style
- ❌ no first class conditions/restarts
- ❌ no dynamic scope

since writing this i had the idea to use dynamic scope as well but i'm not sure how to do it yet. thinking about a compositional structure on environments that lets you merge only the fluid parts linearly

## setup

In [ ]:
import PIL
import os
import sys
import math
import time
import json
import hashlib
import IPython

import dataclasses
import functools as fn
import numpy
# import jax
# import jax.numpy as jnp

# import requests

from typing import Any
from typing import Tuple
from typing import Optional
from typing import Union
from typing import List
from typing import Callable
from typing import Dict
# from typing import override

In [ ]:
class Object:
  def _internal(self, name: str, *xs: 'Object') -> 'Condition':
    args = ' '.join([str(self)] + [str(x) for x in list(xs)])
    return Condition(f'''
Lisp attempted to call the internal procedure {name} on \
the following arguments:

({args})

However, these arguments are invalid for the procedure {name}.
'''.strip())

  def _expected(self, expected: str) -> 'Condition':
    return Condition(f'''
Expected an object with tag `{expected}`, but got `{self}`.
'''.strip())

  def eval(
    self,
    env: 'Environment',
    ctx: 'Context',
  ) -> 'Object':
    return ctx(self)

  def evlis(
    self,
    env: 'Environment',
    ctx: 'Context',
  ) -> 'Object':
    raise self._internal('__evlis__')

  def exec(
    self,
    env: 'Environment',
    ctx: 'Context',
  ) -> 'Object':
    raise self._internal('__exec__')

  def bind(self, rhs: 'Object', env: 'Environment'):
    raise self._internal('__bind__', rhs)

  def __eq__(self, rhs: 'Object') -> bool:
    return self is rhs

  def __add__(self, rhs: 'Object') -> 'Object':
    raise self._internal('__add__', rhs)

  def __mul__(self, rhs: 'Object') -> 'Object':
    raise self._internal('__mul__', rhs)

  def __sub__(self, rhs: 'Object') -> 'Object':
    raise self._internal('__sub__', rhs)

  def __truediv__(self, rhs: 'Object') -> 'Object':
    raise self._internal('__div__', rhs)

  def __len__(self) -> int:
    raise self._internal('__len__')

  def assert_nil(self) -> 'Object':
    raise self._expected('()')

  def assert_pair(self) -> 'Object':
    raise self._expected('pair')

  def assert_symbol(self) -> 'Object':
    raise self._expected('symbol')

  def assert_constant(self) -> 'Object':
    raise self._expected('constant')

  def assert_variable(self) -> 'Object':
    raise self._expected('variable')

  def assert_string(self) -> 'Object':
    raise self._expected('string')

  def assert_boolean(self) -> 'Object':
    return self._expected('boolean')

  def assert_scalar(self) -> 'Object':
    raise self._expected('scalar')

  def assert_tensor(self) -> 'Object':
    raise self._expected('tensor')
  
  def assert_environment(self) -> 'Object':
    raise self._expected('environment')

  def assert_procedure(self) -> 'Object':
    raise self._expected('procedure')

  def assert_abstract(self) -> 'Object':
    raise self._expected('abstract')

  def assert_evaluate(self) -> 'Object':
    raise self._expected('evaluate')

  def assert_builtin(self) -> 'Object':
    raise self._expected('builtin')

  def assert_truthy(self) -> 'Object':
    return self

  @property
  def is_truthy(self) -> bool:
    return True

  @property
  def is_list(self) -> bool:
    return False

  @staticmethod
  def to_lisp_list(buf: List['Object']) -> 'Object':
    xs = Nil()
    for obj in reversed(buf):
      xs = Pair(obj, xs)
    return xs

  @staticmethod
  def to_python_list(xs: 'Object') -> List['Object']:
    assert xs.is_list
    buf = []
    while isinstance(xs, Pair):
      buf.append(xs.fst)
      xs = xs.snd
    return buf

  @staticmethod
  def read(source: str) -> List['Object']:
    stack = []
    build = []
    index = 0
    while index < len(source):
      if source[index] == '(':
        stack.append(build)
        build = []
        index += 1
      elif source[index] == ')':
        if len(stack) == 0:
          raise Condition(f'''
There were unbalanced parentheses while reading the following code:

{source}
'''.strip(), source)
        value = Object.to_lisp_list(build)
        build = stack.pop()
        build.append(value)
        index += 1
      elif source[index] == '"':
        index += 1
        start = index
        while index < len(source):
          if source[index] == '"':
            break
          index += 1
        if index >= len(source):
          raise Condition(f'''
There were unbalanced quotes while reading the following code:

{source}
'''.strip(), source)
        body = source[start:index]
        build.append(String(body))
        index += 1
      elif is_whitespace(source[index]):
        while index < len(source):
          if not is_whitespace(source[index]):
            break
          index += 1
      else:
        start = index
        while index < len(source):
          if is_separator(source[index]):
            break
          index += 1
        body = source[start:index]
        if body.startswith('#<'):
          raise Condition(f'''
The unknown token `{body}` was encountered while reading \
the following code:

{source}
'''.strip(), source)
        # elif body == '#':
          # build.append(Nil())
        elif body.startswith('#'):
          build.append(Constant(body))
        else:
          try:
            build.append(Scalar(float(body)))
          except ValueError:
            build.append(Variable(body))
    if len(stack) > 0:
      raise Condition(f'''
There were unbalanced parentheses while reading the following code:

{source}
'''.strip(), source)
    return build

Context = Callable[[Object], Object]

class Environment(Object):
  body: Dict[str, Object]
  next: Optional[Object]

  def __init__(
    self,
    next: Optional['Environment'] = None,
  ):
    self.body = {}
    self.next = next

  def assert_environment(self) -> Object:
    return self

  def __call__(
    self,
    args: Object,
    env: 'Environment',
    ctx: Context,
  ) -> Object:
    assert isinstance(args, Pair)
    if len(args) != 1:
      raise Condition(f'''
An environment maps symbols to objects, but this environment \
was provided with the following arguments:

{args}
'''.strip())
    return ctx(self[args.fst])

  def __getitem__(
    self,
    key: Union[str, Object],
  ) -> Object:
    if isinstance(key, Constant):
      key = key.name
    if isinstance(key, Variable):
      key = key.name
    assert isinstance(key, str)
    if key.startswith('#'):
      return _get_constant(key)
    if key in self.body:
      return self.body[key]
    if self.next is not None:
      return self.next[key]
    raise Condition(f'''
The variable `{key}`  is undefined.
'''.strip())

  def __setitem__(
    self,
    key: Union[str, Object],
    value: Object,
  ):
    if isinstance(key, Constant):
      key = key.name
    if isinstance(key, Variable):
      key = key.name
    assert isinstance(key, str)
    assert not key.startswith('#')
    if key in self.body:
      raise Condition(f'''
The variable `{key}` cannot be defined as `{value}` \
because it is already defined as `{self.body[key]}`.
'''.strip())
    self.body[key] = value

  def __contains__(
    self,
    key: Union[str, Object],
  ) -> bool:
    if isinstance(key, Constant):
      key = key.name
    if isinstance(key, Variable):
      key = key.name
    assert isinstance(key, str)
    if key in self.body:
      return True
    if self.next is not None:
      return key in self.next
    return False

  def __str__(self) -> str:
    return f'#<environment>'

@dataclasses.dataclass
class Nil(Object):
  def assert_nil(self) -> Object:
    return self

  def assert_truthy(self) -> Object:
    raise self._expected('truthy')
  
  def bind(self, rhs, env):
    if not isinstance(rhs, Nil):
      super().bind(rhs, env)

  def evlis(self, env: Environment, ctx: Context) -> Object:
    return ctx(self)

  def exec(self, env: Environment, ctx: Context) -> Object:
    return ctx(self)

  @property
  def is_truthy(self) -> bool:
    return False

  @property
  def is_list(self) -> bool:
    return True

  def __add__(self, rhs: Object) -> bool:
    if rhs.is_list:
      return rhs
    raise self._internal('__add__', rhs)

  def __eq__(self, rhs: Object) -> bool:
    return isinstance(rhs, Nil)

  def __len__(self) -> int:
    return 0

  def __str__(self) -> str:
    return '()'

@dataclasses.dataclass
class Pair(Object):
  fst: Object
  snd: Object

  def assert_pair(self) -> Object:
    return self

  def bind(self, rhs, env):
    if not isinstance(rhs, Pair):
      return super().bind(rhs, env)
    self.fst.bind(rhs.fst, env)
    self.snd.bind(rhs.snd, env)

  def eval(self, env: Environment, ctx: Context) -> Object:
    def proc_ctx(proc):
      return proc(self.snd, env, ctx)
    return self.fst.eval(env, proc_ctx)

  def evlis(self, env: Environment, ctx: Context) -> Object:
    def fst_ctx(fst):
      def snd_ctx(snd):
        return ctx(Pair(fst, snd))
      return self.snd.evlis(env, snd_ctx)
    return self.fst.eval(env, fst_ctx)

  def exec(self, env: Environment, ctx: Context) -> Object:
    def fst_ctx(fst):
      def snd_ctx(snd):
        if isinstance(snd, Nil):
          return ctx(fst)
        return ctx(snd)
      return self.snd.exec(env, snd_ctx)
    return self.fst.eval(env, fst_ctx)

  @property
  def is_list(self) -> bool:
    return self.snd.is_list

  def __add__(self, rhs: Object) -> Object:
    if not self.is_list or not rhs.is_list:
      raise self._internal('__add__', rhs)
    if isinstance(rhs, Nil):
      return self
    stack = []
    xs = self
    while not isinstance(xs, Nil):
      assert isinstance(xs, Pair)
      stack.append(xs.fst)
      xs = xs.snd
    xs = rhs
    for value in reversed(stack):
      xs = Pair(value, xs)
    return xs

  def __eq__(self, rhs: Object) -> bool:
    if not isinstance(rhs, Pair):
      return False
    return self.fst == rhs.fst and self.snd == rhs.snd

  def __len__(self) -> int:
    return 1+len(self.snd)

  def __str__(self) -> str:
    if self.is_list:
      xs = Object.to_python_list(self)
      strings = [f'{obj}' for obj in xs]
      content = ' '.join(strings)
      return f'({content})'
    return f'(#p {self.fst} {self.snd})'

@dataclasses.dataclass
class Symbol(Object):
  def assert_symbol(self) -> Object:
    return self

@dataclasses.dataclass
class Constant(Symbol):
  name: str

  def assert_constant(self) -> Object:
    return self
  
  def eval(self, env: Environment, ctx: Context) -> Object:
    return ctx(env[self])

  def __eq__(self, rhs: Object) -> bool:
    if not isinstance(rhs, Constant):
      return False
    return self.name == rhs.name
  
  def __len__(self) -> int:
    return len(self.name)

  def __str__(self) -> str:
    return self.name

@dataclasses.dataclass
class Variable(Symbol):
  name: str

  def assert_variable(self) -> Object:
    return self
  
  def bind(self, rhs, env):
    env[self] = rhs

  def eval(self, env: Environment, ctx: Context) -> Object:
    return ctx(env[self])

  def __eq__(self, rhs: Object) -> bool:
    if not isinstance(rhs, Variable):
      return False
    return self.name == rhs.name

  def __len__(self) -> int:
    return len(self.name)

  def __str__(self) -> str:
    return self.name

@dataclasses.dataclass
class Boolean(Object):
  value: bool

  def assert_boolean(self) -> Object:
    return self

  def assert_truthy(self) -> Object:
    if self.value:
      return self
    raise self._expected('truthy')

  @property
  def is_truthy(self) -> bool:
    return self.value

  def __eq__(self, rhs: Object) -> bool:
    if not isinstance(rhs, Boolean):
      return False
    return self.value == rhs.value

  def __str__(self) -> str:
    return '#t' if self.value else '#f'

@dataclasses.dataclass
class Scalar(Object):
  value: float

  def assert_scalar(self) -> Object:
    return self
  
  def __add__(self, rhs: Object) -> Object:
    if not isinstance(rhs, Scalar):
      raise self._internal('__add__', rhs)
    return Scalar(self.value+rhs.value)

  def __mul__(self, rhs: Object) -> Object:
    if not isinstance(rhs, Scalar):
      raise self._internal('__mul__', rhs)
    return Scalar(self.value*rhs.value)

  def __sub__(self, rhs: Object) -> Object:
    if not isinstance(rhs, Scalar):
      raise self._internal('__sub__', rhs)
    return Scalar(self.value-rhs.value)

  def __truediv__(self, rhs: Object) -> Object:
    if not isinstance(rhs, Scalar) or rhs.value == 0:
      raise self._internal('__div__', rhs)
    return Scalar(self.value/rhs.value)

  def __eq__(self, rhs: Object) -> bool:
    if not isinstance(rhs, Scalar):
      return False
    return self.value == rhs.value
  
  def __str__(self) -> str:
    return f'{self.value}'

@dataclasses.dataclass
class Tensor(Object):
  value: numpy.ndarray

  def assert_tensor(self) -> Object:
    return self
  
  def __add__(self, rhs: Object) -> Object:
    if not isinstance(rhs, Tensor):
      raise self._internal('__add__', rhs)
    return Tensor(self.value+rhs.value)

  def __mul__(self, rhs: Object) -> Object:
    if not isinstance(rhs, Tensor):
      raise self._internal('__mul__', rhs)
    return Tensor(self.value*rhs.value)

  def __sub__(self, rhs: Object) -> Object:
    if not isinstance(rhs, Tensor):
      raise self._internal('__sub__', rhs)
    return Tensor(self.value-rhs.value)

  def __truediv__(self, rhs: Object) -> Object:
    if not isinstance(rhs, Tensor):
      raise self._internal('__div__', rhs)
    return Tensor(self.value/rhs.value)
  
  def __eq__(self, rhs: Object) -> bool:
    if not isinstance(rhs, Tensor):
      return False
    return self.value == rhs.value
  
  def __str__(self) -> str:
    return f'#<tensor>'

@dataclasses.dataclass
class String(Object):
  value: str

  def assert_string(self) -> Object:
    return self
  
  def __add__(self, rhs: Object) -> Object:
    if not isinstance(rhs, String):
      raise self._internal('__add__', rhs)
    return String(self.value+rhs.value)
  
  def __eq__(self, rhs: Object) -> bool:
    if not isinstance(rhs, String):
      return False
    return self.value == rhs.value
  
  def __len__(self) -> int:
    return len(self.value)

  def __str__(self) -> str:
    return f'"{self.value}"'

def _get_constant(key: str) -> Object:
  if key == '#t':
    return Boolean(True)
  if key == '#f':
    return Boolean(False)
  if key == '#p':
    return Evaluate(ProcPair())

@dataclasses.dataclass
class Procedure(Object):
  def assert_procedure(self) -> Object:
    return self

  def __call__(
    self,
    args: Object,
    env: Environment,
    ctx: Context,
  ) -> Object:
    return ctx(args)

  def _condition(
    self,
    args: Object,
    restart: List[Dict],
  ) -> Exception:
    message = f'''
There was an error during execution of the following procedure:

{self.description}

The procedure was called with the following arguments:

{args}
'''.strip()
    return Condition(message=message, restart=restart)

  def _retry(self, env, ctx) -> List[Dict]:
    return {
      'description': '''
Call the procedure again with different arguments.
'''.strip(),
      'context': lambda x: self(x, env, ctx),
    }

@dataclasses.dataclass
class Builtin(Procedure):
  @property
  def name(self) -> str:
    return '__builtin__'

  @property
  def description(self) -> str:
    return '__description__'

  def assert_builtin(self) -> Object:
    return self

  def __call__(
    self,
    args: Object,
    env: Environment,
    ctx: Context,
  ) -> Object:
    return ctx(args)

  def __str__(self) -> str:
    return '#<procedure>'

@dataclasses.dataclass
class Abstract(Procedure):
  head: Object
  body: Object
  dynamic: Object
  lexical: Environment

  def assert_abstract(self) -> Object:
    return self
  
  def __call__(
    self,
    args: Object,
    env: Environment,
    ctx: Context,
  ) -> Object:
    local_env = Environment(self.lexical)
    self.head.bind(args, local_env)
    self.dynamic.bind(env, local_env)
    return self.body.exec(local_env, ctx)

  def __str__(self) -> str:
    return '#<procedure>'

@dataclasses.dataclass
class Evaluate(Procedure):
  body: Object

  def assert_evaluate(self) -> Object:
    return self
  
  def __call__(
    self,
    args: Object,
    env: Environment,
    ctx: Context,
  ) -> Object:
    def args_ctx(args):
      return self.body(args, env, ctx)
    return args.evlis(env, args_ctx)

  def __str__(self) -> str:
    return '#<procedure>'

@dataclasses.dataclass
class Condition(Exception):
  message: str
  restart: List[Dict]

  def __init__(
    self,
    message: str,
    restart: Optional[List[Dict]] = None,
  ):
    self.message = message
    self.restart = restart or []

  def append(self, restart: List[Dict]) -> 'Condition':
    return Condition(self.message, self.restart+restart)
  
  def __str__(self) -> str:
    return f'{self.message}'

def is_separator(token: str) -> bool:
  return token in ' \t\r\n()"'

def is_whitespace(token: str) -> bool:
  return token in ' \t\r\n'

In [ ]:
@dataclasses.dataclass
class ProcUnop(Builtin):
  _name: str
  _desc: str
  _body: Callable[[Object], Object]

  def __init__(
    self,
    name: str,
    desc: str,
    body: Callable[[Object], Object],
  ):
    self._name = name
    self._desc = desc
    self._body = body

  @property
  def name(self) -> str:
    return self._name

  @property
  def description(self) -> str:
    return self._desc
    
  def __call__(self, args, env, ctx):
    if len(args) != 1:
      raise self._condition(args, [self._retry(env, ctx)])
    fst = args.fst
    try:
      dst = self._body(fst)
    except Condition as err:
      raise err.append([self._retry(env, ctx)])
    return ctx(dst)

@dataclasses.dataclass
class ProcBinop(Builtin):
  _name: str
  _desc: str
  _body: Callable[[Object], Object]

  def __init__(
    self,
    name: str,
    desc: str,
    body: Callable[[Object], Object],
  ):
    self._name = name
    self._desc = desc
    self._body = body

  @property
  def name(self) -> str:
    return self._name

  @property
  def description(self) -> str:
    return self._desc

  def __call__(self, args, env, ctx):
    if len(args) != 2:
      raise self._condition(args, [self._retry(env, ctx)])
    fst = args.fst
    snd = args.snd.fst
    try:
      dst = self._body(fst, snd)
    except Condition as err:
      raise err.append([self._retry(env, ctx)])
    return ctx(dst)

@dataclasses.dataclass
class ProcAnd(Builtin):
  @property
  def name(self) -> str:
    return 'and'

  @property
  def description(self) -> str:
    return f'''
({self.name} VALUE...)

Evaluates each of its arguments one at a time from left to right.
If an expression evaluates to a falsy value, this procedure returns\
that value and doesn't evaluate any of the other expressions;\
otherwise it returns the value of the last expression.
If this procedure is called with no arguments it will return #t.
'''.strip()

  def __call__(self, args, env, ctx):
    return self.__call(args, env, ctx, Boolean(True))

  def __call(self, args, env, ctx, prev):
    def fst_ctx(fst):
      if not fst.is_truthy:
        return ctx(fst)
      return self.__call(args.snd, env, ctx, fst)
    if isinstance(args, Nil):
      return ctx(prev)
    assert isinstance(args, Pair)
    return args.fst.eval(env, fst_ctx)

@dataclasses.dataclass
class ProcOr(Builtin):
  @property
  def name(self) -> str:
    return 'or'

  @property
  def description(self) -> str:
    return f'''
({self.name} VALUE...)

Evaluates each of its arguments one at a time from left to right.
If an expression evaluates to a truthy value, this procedure returns \
that value and doesn't evaluate any of the other expressions; \
otherwise it returns the value of the last expression.
If this procedure is called with no arguments it will return #f.
'''.strip()

  def __call__(self, args, env, ctx):
    return self.__call(args, env, ctx, Boolean(False))

  def __call(self, args, env, ctx, prev):
    def fst_ctx(fst):
      if fst.is_truthy:
        return ctx(fst)
      return self.__call(args.snd, env, ctx, fst)
    if isinstance(args, Nil):
      return ctx(prev)
    assert isinstance(args, Pair)
    return args.fst.eval(env, fst_ctx)

@dataclasses.dataclass
class ProcIf(Builtin):
  @property
  def name(self) -> str:
    return 'if'

  @property
  def description(self) -> str:
    return f'''
({self.name} TEST THEN [ELSE])

Evaluates TEST; if that value is truthy, then evaluates THEN; \
if not then evaluates ELSE if it is provided, otherwise returns ().
'''.strip()

  def __call__(self, args, env, ctx):
    if len(args) in [0, 1] or len(args) > 3:
      raise self._condition(args, [self._retry(env, ctx)])
    def cond_ctx(cond):
      if cond.is_truthy:
        target = args.snd.fst
        return target.eval(env, ctx)
      if len(args) == 3:
        target = args.snd.snd.fst
        return target.eval(env, ctx)
      target = Nil()
      return ctx(target)
    cond = args.fst
    return cond.eval(env, cond_ctx)

@dataclasses.dataclass
class ProcFold(Builtin):
  _name: str
  _desc: str
  _body: Callable[[Object, Object], Object]

  def __init__(
    self,
    name: str,
    desc: str,
    body: Callable[[Object, Object], Object],
  ):
    self._name = name
    self._desc = desc
    self._body = body

  @property
  def name(self) -> str:
    return self._name

  @property
  def description(self) -> str:
    return self._desc

  def __call__(self, args, env, ctx):
    state = None
    xs = args
    while not isinstance(xs, Nil):
      assert isinstance(xs, Pair)
      if state is None:
        state = xs.fst
      else:
        try:
          state = self._body(state, xs.fst)
        except Condition as err:
          raise err.append([self._retry(env, ctx)])
      xs = xs.snd
    if state is None:
      raise self._condition(args, [self._retry(env, ctx)])
    return ctx(state)

@dataclasses.dataclass
class ProcVau(Builtin):
  @property
  def name(self) -> str:
    return 'vau'

  @property
  def description(self) -> str:
    return f'''
({self.name} PARAMS DYNAMIC-ENV BODY...)

Return a vau expression.
'''.strip()

  def __call__(self, args, env, ctx):
    if len(args) < 3:
      raise self._condition(args, [self._retry(env, ctx)])
    head = args.fst
    body = args.snd.snd
    dynamic = args.snd.fst
    lexical = env
    target = Abstract(head, body, dynamic, lexical)
    return ctx(target)

@dataclasses.dataclass
class ProcEval(Builtin):
  @property
  def name(self) -> str:
    return 'eval'

  @property
  def description(self) -> str:
    return f'''
({self.name} EXPR ENV)

Evaluate the expression EXPR in the environment ENV.
'''.strip()

  def __call__(self, args, env, ctx):
    if len(args) != 2:
      raise self._condition(args, [self._retry(env, ctx)])
    exp = args.fst
    dst = args.snd.fst
    def dst_ctx(dst):
      if not isinstance(dst, Environment):
        raise self._condition(args, [self._retry(env, ctx)])
      return exp.eval(dst, ctx)
    return dst.eval(env, dst_ctx)

@dataclasses.dataclass
class ProcDefine(Builtin):
  @property
  def name(self) -> str:
    return 'define'

  @property
  def description(self) -> str:
    return f'''
({self.name} NAME EXPR)

Associate the symbol NAME with the value of EXPR \
in the current environment.
'''.strip()

  def __call__(self, args, env, ctx):
    if len(args) != 2 or not isinstance(args.fst, Variable):
      raise self._condition(args, [self._retry(env, ctx)])
    lhs = args.fst
    rhs = args.snd.fst
    def rhs_ctx(rhs):
      lhs.bind(rhs, env)
      target = Nil()
      return target
    return rhs.eval(env, rhs_ctx)

@dataclasses.dataclass
class ProcCond(Builtin):
  @property
  def name(self) -> str:
    return 'cond'

  @property
  def description(self) -> str:
    return f'''
({self.name} TEST EXPR...)

Takes a list of TEST/EXPR pairs. It evaluates each TEST one at a \
time, from left to right.  If a test returns a truthy value, \
evaluates and returns the value of the corresponding expr and \
doesn't evaluate any of the other tests or exprs.
({self.name}) returns ().
'''.strip()

  def __call__(self, args, env, ctx):
    return self.__call(args, env, ctx, args)

  def __call(self, args, env, ctx, init):
    if len(args) == 0:
      target = Nil()
      return ctx(target)
    if len(args) < 2:
      raise self._condition(init, [self._retry(env, ctx)])
    fst = args.fst
    snd = args.snd.fst
    def fst_ctx(fst):
      if fst.is_truthy:
        return snd.eval(env, ctx)
      rest = args.snd.snd
      return self.__call(rest, env, ctx, init)
    return fst.eval(env, fst_ctx)

@dataclasses.dataclass
class ProcEqual(Builtin):
  @property
  def name(self) -> str:
    return '='

  @property
  def description(self) -> str:
    return f'''
({self.name} VALUE...)

Predicates structural equality of every VALUE. ({self.name}) is #t.
'''.strip()
  
  def __call__(self, args, env, ctx):
    state = None
    while not isinstance(args, Nil):
      assert isinstance(args, Pair)
      if state is None:
        state = args.fst
      elif not state == args.fst:
        target = Boolean(False)
        return ctx(target)
      state = args.fst
      args = args.snd
    target = Boolean(True)
    return ctx(target)

@dataclasses.dataclass
class ProcList(Builtin):
  @property
  def name(self) -> str:
    return 'list'

  @property
  def description(self) -> str:
    return f'''
({self.name} VALUE...)

Returns the list VALUE...
'''.strip()
  
  def __call__(self, args, env, ctx):
    return ctx(args)

def standard_environment() -> Environment:
  env = Environment()

  env['vau'] = ProcVau()
  env['define'] = ProcDefine()

  env['assert'] = Evaluate(ProcUnop(
    name='assert',
    desc=f'''
(assert VALUE)

Signals a condition if VALUE is not truthy.
'''.strip(),
    body=lambda x: x.assert_truthy(),
))

  env['wrap'] = Evaluate(ProcUnop(
    name='wrap',
    desc='''
(wrap PROCEDURE)

Returns a procedure that induces argument evaluation before \
passing its arguments to PROCEDURE.
'''.strip(),
    body=lambda x: Evaluate(x.assert_procedure()),
))

  env['unwrap'] = Evaluate(ProcUnop(
    name='unwrap',
    desc='''
(unwrap PROCEDURE)

Removes a procedure from a wrapper that induces argument evaluation.
'''.strip(),
    body=lambda x: x.assert_evaluate().body,
))

  env['pair'] = Evaluate(ProcBinop(
    name='pair',
    desc='''
(pair FST SND)

Maps two objects to a pair.
'''.strip(),
    body=lambda x,y: Pair(x,y),
))

  env['fst'] = Evaluate(ProcUnop(
    name='fst',
    desc='''
(fst PAIR)

Returns the first element of a pair.
'''.strip(),
    body=lambda x: x.fst
))

  env['snd'] = Evaluate(ProcUnop(
    name='snd',
    desc='''
(snd PAIR)

Returns the second element of a pair.
'''.strip(),
    body=lambda x: x.snd
))

  env['if'] = ProcIf()
  env['cond'] = ProcCond()

  # todo: could these be some sort of evaluating fold?
  env['and'] = ProcAnd()
  env['or'] = ProcOr()

  env['not'] = Evaluate(ProcUnop(
    name='not',
    desc='''
Returns #t if its argument is falsy, and #f otherwise.
'''.strip(),
    body=lambda x: Boolean(False) if x.is_truthy else Boolean(True),
))

  env['='] = Evaluate(ProcEqual())

  env['list'] = Evaluate(ProcList())

  env['+'] = Evaluate(ProcFold(
    name='+',
    desc='''
(+ VALUE...)

Folds a nonempty list of arguments with a different operation \
depending on their type.

For scalars, + is addition.
For tensors, + is pointwise addition.
For strings, + is catenation.
For lists, + is catenation.

Raises a condition if there are no arguments.
'''.strip(),
    body=lambda x, y: x+y
))

  env['*'] = Evaluate(ProcFold(
    name='*',
    desc='''
(* VALUE...)

Folds a nonempty list of arguments with a different operation \
depending on their type.

For scalars, * is multiplication.
For tensors, * is pointwise multiplication.

Raises a condition if there are no arguments.
'''.strip(),
    body=lambda x,y: x*y
))

  env['-'] = Evaluate(ProcFold(
    name='-',
    desc='''
(- VALUE...)

Folds a nonempty list of arguments with a different operation \
depending on their type.

For scalars, - is subtraction.
For tensors, - is pointwise subtraction.

Raises a condition if there are no arguments.
'''.strip(),
    body=lambda x,y: x-y
))

  env['/'] = Evaluate(ProcFold(
    name='/',
    desc='''
(/ VALUE...)

Folds a nonempty list of arguments with a different operation \
depending on their type.

For scalars, / is division.
For tensors, / is pointwise division.

Raises a condition if there are no arguments.
'''.strip(),
    body=lambda x,y: x/y
))

  return env

## test

In [ ]:
buf = Object.read('''
(define foo "Hello, world.")
(assert (= foo "Hello, world."))
(assert (= ((wrap (vau (x) e x)) foo) "Hello, world."))
(assert (= (fst (pair "foo" "bar")) "foo"))
(assert (= (snd (pair "foo" "bar")) "bar"))
(assert (= (not ()) #t))
(assert (= (not #t) #f))
(assert (= (not (list 1 2 3 4)) #f))
(assert
  (= (cond
       #f 0
       #f "Hello, world."
       #t "Goodnight, moon.")
     "Goodnight, moon."))
(assert (= (list 1 2 3 4) (list 1 2 3 4)))
(assert (= (+ 1 2 3 4) 10))
(assert (= (+ "Hello, " "world.") "Hello, world."))
(assert (= (+ (list 1 2 3 4) (list 5 6 7 8)) (list 1 2 3 4 5 6 7 8)))
'''.strip())

env = standard_environment()
for source in buf:
  target = source.eval(env, lambda x: x)
  print(target)

()
#t
#t
#t
#t
#t
#t
#t
#t
#t
#t
#t
#t
